# 0. 폴더 생성 및 데이터 다운로드

- 모든 resource 데이터는 rsc 폴더에 저장
- rsc 폴더 생성

In [ ]:
mkdir rsc

- AI Hub 문서요약텍스트 데이터 다운 : https://aihub.or.kr/aidata/8054
- AI Hub 한국어대화요약 데이터 다운 : https://aihub.or.kr/aidata/30714
- 각각 데이터 다운로드 한 후, 적당한 경로에 저장
  - 문서요약텍스트의 경우 : rsc/Data/Training/AI_hub/
      - 신문기사, 법률문서, 사설잡지
  - 한국어대화요약의 경우 : rsc/Data/Training/AI_hub/한국어대화요약/      
      - 개인및관게, 미용과건강, 상거래(쇼핑), 시사교육, 식음료, 여가생활, 일과직업, 주거와생활, 행사

rsc/Data/Training/AI_hub  
&nbsp;&nbsp;&nbsp;&nbsp;⌞ 법률문서     
&nbsp;&nbsp;&nbsp;&nbsp;⌞ 신문기사         
&nbsp;&nbsp;&nbsp;&nbsp;⌞ 사설잡지           
&nbsp;&nbsp;&nbsp;&nbsp;⌞ 한국어대화요약         
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;⌞ 개인및관계     
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;⌞ 미용과건강         
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;⌞ 등등...

# 1. AI-Hub 신문기사, 법률문서, 사설잡지 데이터 정제

++ 다운받은 데이터 압축 풀어서 저장하기

1. `pandas`, `json` import

In [ ]:
import pandas as pd
import json

2. aihub 에서 다운받은 `.json 파일` 로드 (aihub에서 다운받은 후, train_original.json으로 파일명 변경)

In [ ]:
with open('/content/drive/MyDrive/BITAmin_nlp/rsc/Data/Training/AI_hub/Training/train_original.json') as f:
    json_data = json.load(f)

3. dataframe `df` 으로 저장

In [ ]:
df = pd.DataFrame(json_data)

4. `df` 출력

In [ ]:
df

,name,delivery_date,documents
0,법률문서 프로젝트,2020-12-23 17:23:13,"{'id': '100004', 'category': '일반행정', 'size': '..."
1,법률문서 프로젝트,2020-12-23 17:23:13,"{'id': '100005', 'category': '세무', 'size': 'sm..."
2,법률문서 프로젝트,2020-12-23 17:23:13,"{'id': '100006', 'category': '세무', 'size': 'sm..."
3,법률문서 프로젝트,2020-12-23 17:23:13,"{'id': '100008', 'category': '세무', 'size': 'sm..."
4,법률문서 프로젝트,2020-12-23 17:23:13,"{'id': '100014', 'category': '세무', 'size': 'sm..."
...,...,...,...
24324,법률문서 프로젝트,2020-12-23 17:23:13,"{'id': '71798', 'category': '민사', 'size': 'sma..."
24325,법률문서 프로젝트,2020-12-23 17:23:13,"{'id': '71816', 'category': '민사', 'size': 'sma..."
24326,법률문서 프로젝트,2020-12-23 17:23:13,"{'id': '71817', 'category': '민사', 'size': 'sma..."
24327,법률문서 프로젝트,2020-12-23 17:23:13,"{'id': '71826', 'category': '민사', 'size': 'sma..."


In [ ]:
import pandas as pd
import json

# 'document' 컬럼에 있는 딕셔너리를 별도의 컬럼으로 확장
if 'document' in df.columns:
    document_df = pd.json_normalize(df['document'])
    df = pd.concat([df.drop(columns=['document']), document_df], axis=1)

# 결과 확인
print("Expanded JSON Data:")
print(df.head())  # 상위 5개 행을 출력합니다. 전체를 보려면 df를 사용하세요.


Expanded JSON Data:
        name        delivery_date  \
0  법률문서 프로젝트  2020-12-23 17:23:13   
1  법률문서 프로젝트  2020-12-23 17:23:13   
2  법률문서 프로젝트  2020-12-23 17:23:13   
3  법률문서 프로젝트  2020-12-23 17:23:13   
4  법률문서 프로젝트  2020-12-23 17:23:13   

                                           documents  
0  {'id': '100004', 'category': '일반행정', 'size': '...  
1  {'id': '100005', 'category': '세무', 'size': 'sm...  
2  {'id': '100006', 'category': '세무', 'size': 'sm...  
3  {'id': '100008', 'category': '세무', 'size': 'sm...  
4  {'id': '100014', 'category': '세무', 'size': 'sm...  


5. `df`에서 필요한 column 만 추출하여 `new_df`에 저장 및 출력

In [ ]:
new_df = df[['id', 'category', 'publish_date', 'text', 'abstractive', 'extractive']]


6. `new_df` 의 `abstractive` column에 요약문이 2개 이상인지 확인 (결과:없음)

In [ ]:
for i in range(len(new_df)):
    if len(new_df.loc[i, 'abstractive']) != 1:
        print(i)

7. text와 abstractive 정제   

     **text** 리스트 중 **sentence**만 추출하여 new_df 의 `text` 에 저장  
     **abstractive** 리스트 중 **첫번째 데이터**만 추출하여 new_df 의 `abstractive` 에 저장

In [ ]:
for i in range(len(new_df)):
    context = pd.DataFrame(new_df.loc[i, 'text'])
    new_df.loc[i, 'text'] = context['sentence'].str.cat()
    new_df.loc[i, 'abstractive'] = new_df.loc[i, 'abstractive'][0]

8. `new_df` 출력

In [ ]:
new_df

9. 결측값 확인

In [ ]:
new_df.isnull().sum()

id                0
category        430
publish_date      0
text              0
abstractive       0
extractive        0
dtype: int64

10. 결측값 있는 행 삭제 후 저장 (text, abstractive 결측값 있는지만 확인 -> 있으면 아래 코드 실행)

In [ ]:
new_df.dropna(subset=['abstractive'], inplace=True) # 'abstractive' or 'text'

11. `new_df` 중 첫번째 text 출력해보기

In [ ]:
new_df['text'][0]

'변근아"작은별 일루와. 모래놀이할건데 양말 신고 가자."14일 오후 1시 오산시 세교동에 위치한 한 전원주택.\'숲이랑 놀자\'라는 작은 팻말이 달린 1층짜리 작은 집 근처로 다가가자 아이들의 활기찬 웃음소리가 밖으로 퍼져나오는 것을 들을 수 있었다.이 곳은 지난해 사립유치원 회계 비리 문제 규탄에 나섰던 동탄 지역 학부모들이 모여 지난 4일 문을 연 공동육아나눔터로 현재 7가족이 모여 아이들을 같이 돌보고 있는 장소기도 하다.당초 학부모 협동조합형 유치원을 운영하려 했으나, 임대 공간 문제로 유치원 운영이 내년 상반기에나 가능해지면서 기다리는 1년여 동안 부모들이 공동육아에 나선 것이다.이날도 엄마 4명이 모여 친구들과 거실을 마음껏 뛰어다니고 있는 아이들을 살갑게 챙기는 모습을 확인할 수 있었다.집안을 들어서자마자 보이는 거실 벽을 비롯해 각 방에는 아이들이 그려놓은 그림과 손바닥에 페인트를 칠해 손도장을 찍은 작품들이 가득했다.현재 \'숲이랑 놀자\'를 다니고 있는 아이들은 총 11명으로 영유아부터 4~7세까지 연령대도 다양하다.가정보육을 받다 온 아이도 있고, 올해 새롭게 유치원에 입학하려다 사립유치원 문제가 터지면서 공동육아나눔터를 찾은 아이도 있다.4살 아이와 태어난 지 얼마 되지 않은 아이를 데리고 공동육아나눔터를 오는 엄모(32·여)씨는 "부모님도 그렇고 그냥 어린이집을 보내라는 말을 많이 했는데, 내가 직접 아이를 돌보고 아이들도 마음껏 뛰어놀 수 있는 장점 등에 공동육아를 선택하게 됐다"고 설명했다.오전 9시에서 9시30분 사이 아이들이 자유롭게 도착하면 시작되는 공동육아는 오후 3시30분까지 진행된다.이 시간동안 학부모들은 아이들과 집 뒤편에 있는 작은 산을 올라 숲체험을 하기도 하고, 집 앞에 꾸며놓은 텃밭을 일구거나 모래밭에 들어가 흙장난도 하는 등 말 그대로 자유롭게 노는 생활을 보내고 있다.이 곳에서 강조되는 것은 \'잘 놀자\' 딱 하나다.무언가를 많이 시키기 보다는 아이들이 하고 싶은 것을 할 수 있도록 지원해주자는 학부모들의 

12. `new_df` 를 `train_v1.csv` 로 저장

In [ ]:
new_df.to_csv('rsc/Data/Training/AI_hub/신문기사/train_v1.csv')

13. `train_v1.csv` 를 다시 읽어 출력 (저장 확인용)

In [ ]:
train_v1 = pd.read_csv('rsc/Data/Training/AI_hub/신문기사/train_v1.csv', index_col = 0)
train_v1

,id,category,publish_date,text,abstractive,extractive
0,337574245,종합,2019-03-14 19:07:12,"변근아""작은별 일루와. 모래놀이할건데 양말 신고 가자.""14일 오후 1시 오산시 세...",사립유치원 회계 비리 문제 규탄에 나섰던 동탄 지역 학부모들 7가족이 모여 지난 4...,"[4, 2, 13]"
1,365817620,경제,2019-11-05 07:38:00,가입연령 60세→55세 하향 추진주택가격 기준 상향방안도 논의대구 가입자 3148·...,"정부가 사망시까지 연금을 지급하고 평생거주를 보장하는 '주택연금'의 가입연령을, 현...","[10, 12, 19]"
2,366267384,경제,2019-11-07 19:41:21,진천 보련·충주 내포긴들 은상증평 통미·음성 신댕이 동상[충청일보 이정규 기자] 충...,충북농협은 농협중앙회가 주관하는 '2회 깨끗하고 아름다운 농촌마을 가꾸기 경진대회'...,"[2, 5, 6]"
3,330083635,종합,2019-01-06 00:00:00,박영재인천지방해양수산청(청장 박경철)은 13인 이상의 여객을 운송하는 여객선(유·도...,인천지방해양수산청은 13인 이상 여객선 선장들의 직무 수행능력 및 사고 발생시 대응...,"[1, 4, 8]"
4,334328294,종합,2019-02-15 12:31:00,35명 도의원 15일 성명 자유한국당 김진태 이종명 김순례 즉각 제명 요구제주특별자...,제주도의회 35명 의원은 15일 5·18광주민주화운동을 폄훼한 자유한국당 소속 김...,"[1, 2, 3]"
...,...,...,...,...,...,...
240963,342904723,종합,2019-04-25 18:05:40,전북도는 24일 고사동 객리단길 내 전주영화도서관&카페에서 '제1회 도란도란 토크콘...,"전북도는, 도지사 공약사업 실천 및 청년들의 아이디어를 도정에 접목시키기 위해, 2...","[0, 1, 4]"
240964,345485588,종합,2019-05-17 11:41:00,정부가 우리 경제 하방리스크가 확대되면서 주요 실물지표 흐름이 두달째 부진한 모습이...,미중 무역갈등이 초래한 세계경제 둔화 및 국제금융시장의 높은 변동성으로 대한민국 경...,"[0, 1, 14]"
240965,339576576,종합,2019-03-31 16:30:50,불법행위 '택시기사 삼진아웃제' 6월 시행[중부매일 이민우 기자] 지난달 충북지역의...,"청주시는 오는 6월부터 승차거부, 부당요금 징수 등 불법행위를 최근 2년 이내 3회...","[2, 4, 7]"
240966,332010887,사회,2019-01-23 21:35:54,"도교육청, 오는 3월 예정내달 15일까지 단원 모집[충청일보 이정규기자] 충북도교육...","충북예술교육을 이끌어 갈 충북청소년국악관현악단이 오는 3월 창단을 목표로 도내 초,...","[3, 4, 10]"


# 2. AI_hub 한국어대화요약 데이터 정제

++ 다운받은 데이터 모두 압출 풀어서 저장하기

## `1. 각 카테고리별 데이터 정제`
    카테고리: 개인및관계, 미용과건강, 상거래(쇼핑), 시사교육, 식음료, 여가생활, 일과직업, 주거와생활, 행사

In [ ]:
import pandas as pd
import json

1. 정제할 카테고리명 파일 리스트 만들기

In [ ]:
category = ['개인및관계', '미용과건강', '상거래(쇼핑)', '시사교육', '식음료', '여가생활', '일과직업', '주거와생활', '행사']

2. aihub 에서 다운받은 `.json 파일` 로드
3. `text`와 `abstractive` column을 가진 새로운 `new_df` 생성
4. new_df의 `text` 에  df의 **body**>**dialogue**>**utterance**를 합친 `text` 저장    
   new_df의 `abstractive` 에  df의 **body**>**summary**를 저장한 `abstractive` 저장
5. `new_df` 중 null 값이 있는지 확인
6. `new_df` 를 `train_v1.csv` 로 저장

In [ ]:
for file in category:
    with open('/content/drive/MyDrive/BITAmin_nlp/rsc/Data/Training/AI_hub/한국어대화요약/Training'+file+'.json') as f:     #2
        json_data = json.load(f)

        df = pd.DataFrame(json_data)
        df = df['data']

        new_df = pd.DataFrame(columns=['text', 'abstractive'])               #3

        for i in range(len(df)):
            new = pd.DataFrame(df[i]['body']['dialogue'])
            text = ' '.join(list(new['utterance']))                          #4
            abstractive = df[i]['body']['summary']                           #5

            new_df.loc[i, 'text'] = text
            new_df.loc[i, 'abstractive'] = abstractive

        print(new_df.isnull().sum())                                         #6

        new_df.to_csv('rsc/Data/Training/AI_hub/한국어대화요약/'+file+'.csv')

In [ ]:
new_df

,text,abstractive
0,금욜에몇시에옴?? 나반차쓰려규하는뎅 움 몰랑 낮에갈겡 오예 >< 야우리에서 뭐먹을까...,금요일에 반차를 쓸 예정이라서 만나자고 하니 사주면 나가고 아니면 늦잠을 자겠다고 한다.
1,커피 마시면서 가나슈 먹을래? 아! 그럴까????? 먹고 들어가쟈 #@이모티콘# 죠...,빨간 기둥에서 만나서 파이홀에 가서 커피와 가나슈를 먹으려고 한다.
2,"낼 9시15분에 섭웨이에서 봐여ㅎ 영화 픽!! 해놓고ㅋㅋ 넵,,! 조심히가요!! 쌤...","내일 9시 15분에 서브웨이(섭웨이)에서 보자고 하였고, 예매권을 사서 내일 840..."
3,아 뷔페 영업중단됐어...엄마랑 20일이후에 빕스가기로계획했었는데 .....ㅠ 진짜...,뷔페 영업 중단으로 인한 계획 취소에 대해 이야기하고 있다.
4,"ㅋㅋㅋㅋㅋㅋㅋㅋ 비가 생각보다 너무 와서 ㅋㅋㅋ 그래두 우선 간다?! 그친거야,,이...",생각보다 비가 너무 많이 오긴 하지만 조리를 신고 석남역으로 가고 있다.
...,...,...
2585,아 #@이름#아 너 토요일은 시간 돼? 아니면 나 토요일에 대구 갈거같긴한데 그때봐...,토요일에 대구에 있을 것 같아서 시간이 되니까 그때 봐도 괜찮을 것 같다.
2586,전 건대.....내리는중 아 이 얼굴로 ㅋㅋㅋㅋ ㅋㅋ모징... 건대로 간다고 내가 ...,건국대(건대)에서 만나기 위해 약속 장소로 가면서 상황을 알린다.
2587,동생은 부산첨인가벙가 오늘 장난아니다 그런듯? 애기때와보고 ㅋㅋㅋ 동생 부산와서 좋...,"동생이 부산에 와서 좋아한다고 말하면서, 일정 중 광안리가 있어서 같은 동네 쪽이면..."
2588,친구셋은 온양에서 온단말야 아침 5시에 공항버스를 탄대 아 그렇구만 난 우리집근처에...,전철 타고 택시 타야 해서 공항 도착하면 진 빠질 것 같아서 밤 비행기로 바꾸고 싶...


## `2. 모든 카테고리 데이터 합치기`

1. 합칠 카테고리 리스트 생성

In [ ]:
category = ['개인및관계', '미용과건강', '상거래(쇼핑)', '시사교육', '식음료', '여가생활', '일과직업', '주거와생활', '행사']

2. 새로운 dataframe `new` 생성

In [ ]:
new = pd.DataFrame()
new

""


3. 각 카테고리별 csv 파일을 저장한 `df`와 `new` 합침 (각 카테고리 이름과 길이 출력)

In [ ]:
for file in category:
    df = pd.read_csv('rsc/Data/Training/AI_hub/한국어대화요약/'+file+'.csv', index_col=0)
    print(file, len(df))
    new = pd.concat([new, df])
    new.reset_index(inplace=True, drop=True)

개인및관계 8926
미용과건강 2065
상거래(쇼핑) 3185
시사교육 1855
식음료 3675
여가생활 4480
일과직업 2519
주거와생활 5705
행사 2590


4. `new` 의 index 초기화

In [ ]:
new.reset_index(inplace=True, drop=True)

5. `new` 출력

In [ ]:
new

,text,abstractive
0,ㅠㅠ퇴근했옹~? 응ㅋㅋㅋㅋ퇴근했지ㅜ 나두30분정도뒤에 퇴근할꺼같앙ㅠ 오♡♡♡♡♡ 우...,퇴근했냐고 물으니 30분 정도 뒤에 퇴근할 것 같다고 하고 둘 다 일찍 간다고 얼른...
1,아주 고와ㅋㅋㅋㅋ 아주 곱다구....????? #@이모티콘# 근데 #@이름#짱 손 ...,엄지손가락에 상처가 있지만 곱다.
2,나 금정빠야 금방정에빠져 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ ...,금방 정에 빠진다고 하니 헤어지는 게 너무 힘들다.
3,아 열받어 ㅎㅎ 난 아.. 저 아우터 넘 충격이야 무슨일이야..? 진짜.....??...,저 아우터는 충격이라고 하자 남자 옷을 어디서 사야 하는지 남자 친구한테 물어봐 달...
4,"#@이름#아 20대때 너무 조급해하지말고, 자신의 가치를 평가절하하지 말고, 계속 ...",성공한 인생은 자기가 좋아하는 걸 찾는 거라고 이야기한다.
...,...,...
34995,아 #@이름#아 너 토요일은 시간 돼? 아니면 나 토요일에 대구 갈거같긴한데 그때봐...,토요일에 대구에 있을 것 같아서 시간이 되니까 그때 봐도 괜찮을 것 같다.
34996,전 건대.....내리는중 아 이 얼굴로 ㅋㅋㅋㅋ ㅋㅋ모징... 건대로 간다고 내가 ...,건국대(건대)에서 만나기 위해 약속 장소로 가면서 상황을 알린다.
34997,동생은 부산첨인가벙가 오늘 장난아니다 그런듯? 애기때와보고 ㅋㅋㅋ 동생 부산와서 좋...,"동생이 부산에 와서 좋아한다고 말하면서, 일정 중 광안리가 있어서 같은 동네 쪽이면..."
34998,친구셋은 온양에서 온단말야 아침 5시에 공항버스를 탄대 아 그렇구만 난 우리집근처에...,전철 타고 택시 타야 해서 공항 도착하면 진 빠질 것 같아서 밤 비행기로 바꾸고 싶...


6. `new_df` 를 `train_v1.csv` 로 저장

In [ ]:
new.to_csv('rsc/Data/Training/AI_hub/한국어대화요약/train_v1.csv')

# 3. Dacon 신문기사 데이터 정제

++ github 오픈소스에 존제하는 dat 폴더 안의 dacon train/test set

1. tar.gz 파일 압출 풀어주기

In [ ]:
!tar -xvf src/KoBART/data/train.tar.gz

train.tsv


2. 압축된 train.tsv 파일 rsc 폴더로 이동

In [ ]:
mv train.tsv rsc/data/Training/Dacon

3. 해당 경로의 tsv 파일을 읽어 `dacon`에 저장

In [ ]:
dacon = pd.read_csv('rsc/Data/Training/Dacon/train.tsv', sep='\t')

4. `dacon` 출력

In [ ]:
dacon

,news,summary
0,일제 강제징용피해자들이 지난 2월 발의된 ‘근로정신대 피해자 지원법률안’의 조속한 ...,13일 광주시의회 시민 소통실에서 시민단체 '근로정신대 할머니와 함께 하는 시민모임...
1,문재인 대통령이 이르면 9일 중폭 개각을 단행할 것으로 보인다. 여권 관계자는 8일...,"문재인 대통령이 이르면 9일 장관 3~4명, 장관급 3명을 교체하는 개각을 단행하고..."
2,전남 어촌마을 정주여건 개선 ‘탄력’ 해수부 공모서 고흥 득량도·강진 신전 등 4개...,"해양수산부 어촌종합개발 거점 개발사업에 전남 고흥 득량도, 강진 신전, 완도 충동,..."
3,여자친구 알몸을 무단으로 촬영해 인터넷 성인 카페에 유포한 남성이 재판에 넘겨졌다....,피해자 A씨는 가해자인 B씨의 엄벌을 촉구하며 국민청원에 피해사실을 공개하였고 결국...
4,대구시의회 배지숙 의장이 개회를 선언하고 있다. 대구시의회(의장 배지숙)는 14일부...,대구시의회가 14일 제1차 본회의를 시작으로 제265회 임시회를 개최하고 ‘대구시시...
...,...,...
34237,"민생 구하기부터 나서야” 자유한국당은 7일 더불어민주당을 향해 ‘재판 불복, 민심 ...",김병준 비상대책위원장은 민주당 이해찬 대표가 대선불복은 절대 받아들일 수 없는 것이...
34238,"민주평화통일자문회의 당진시협의회(회장 김광일, 이하 민주평통) 한성희 수석부회장이 ...","지난 20일, 한성희 수석부회장이 국민훈장을 수상했고, 앞서 지난 19일에는 박병원..."
34239,충남도서관 전경 충남도서관이 한국문화공간건축학회로부터 도서관 부문 '2018 한국문...,"올해로 4회째를 맞는 한국문화공간상에서 충남 도서관이 공공성과 접근성, 사용자들의 ..."
34240,전남도의회는 10일 열린 제 335회 임시회 본회의에서 이광일(더불어민주당·여수 1...,전남도의회는 헌재가 현행 해상경계선을 인정하고 현행 해상경계선을 경계선으로 확정할 ...


5. `dacon` 의 **news**-> **text**, **summary**-> **abstractive**로 column의 이름 수정 및 출력

In [ ]:
dacon.rename(columns={'news':'text', 'summary':'abstractive'}, inplace=True)
dacon

,text,abstractive
0,일제 강제징용피해자들이 지난 2월 발의된 ‘근로정신대 피해자 지원법률안’의 조속한 ...,13일 광주시의회 시민 소통실에서 시민단체 '근로정신대 할머니와 함께 하는 시민모임...
1,문재인 대통령이 이르면 9일 중폭 개각을 단행할 것으로 보인다. 여권 관계자는 8일...,"문재인 대통령이 이르면 9일 장관 3~4명, 장관급 3명을 교체하는 개각을 단행하고..."
2,전남 어촌마을 정주여건 개선 ‘탄력’ 해수부 공모서 고흥 득량도·강진 신전 등 4개...,"해양수산부 어촌종합개발 거점 개발사업에 전남 고흥 득량도, 강진 신전, 완도 충동,..."
3,여자친구 알몸을 무단으로 촬영해 인터넷 성인 카페에 유포한 남성이 재판에 넘겨졌다....,피해자 A씨는 가해자인 B씨의 엄벌을 촉구하며 국민청원에 피해사실을 공개하였고 결국...
4,대구시의회 배지숙 의장이 개회를 선언하고 있다. 대구시의회(의장 배지숙)는 14일부...,대구시의회가 14일 제1차 본회의를 시작으로 제265회 임시회를 개최하고 ‘대구시시...
...,...,...
34237,"민생 구하기부터 나서야” 자유한국당은 7일 더불어민주당을 향해 ‘재판 불복, 민심 ...",김병준 비상대책위원장은 민주당 이해찬 대표가 대선불복은 절대 받아들일 수 없는 것이...
34238,"민주평화통일자문회의 당진시협의회(회장 김광일, 이하 민주평통) 한성희 수석부회장이 ...","지난 20일, 한성희 수석부회장이 국민훈장을 수상했고, 앞서 지난 19일에는 박병원..."
34239,충남도서관 전경 충남도서관이 한국문화공간건축학회로부터 도서관 부문 '2018 한국문...,"올해로 4회째를 맞는 한국문화공간상에서 충남 도서관이 공공성과 접근성, 사용자들의 ..."
34240,전남도의회는 10일 열린 제 335회 임시회 본회의에서 이광일(더불어민주당·여수 1...,전남도의회는 헌재가 현행 해상경계선을 인정하고 현행 해상경계선을 경계선으로 확정할 ...


6. `dacon` 을 `train_v1.csv` 로 저장

In [ ]:
dacon.to_csv('rsc/Data/Training/Dacon/train_v1.csv')

7. `train_v1.csv` 를 다시 읽어 출력 (저장 확인용)

In [ ]:
train_v1 = pd.read_csv('rsc/Data/Training/Dacon/train_v1.csv', index_col = 0)
train_v1

,text,abstractive
0,계약금 10%·중도금무이자융자·6개월 후 전매가능 기자 아파트 분양시장이 실수요자 ...,태왕의 '성당 태왕아너스 메트로'모델하우스는 초역세권 입지와 변화하는 라이프스타일에...
1,▲ 박동우 안양동안경찰서 경비교통과 경장 이른 새벽 옆방에서 장난감 소리가 들려온다...,최근 늘어나고 있는 집회시위에서 소음이 문제가 되고 있는데 현행 집회와 시위에 관한...
2,"지난 19일 옛 망주초등학교에서 송귀근 고흥군수와 송우섭 군의회 의장, 귀농·귀촌인...",지난 19일 고흥군에 따르면 송귀근 군수와 주민 등 300여 명이 참석한 가운데 귀...
3,여자 하이다이빙 금메달을 차지한 호주 이프랜드(오른쪽) 선수가 환호하고 있다. /김...,23일 조선대학교에서 열린 여자 하이다이빙 최종전에서 호주의 이프랜드가 금메달을 차...
4,"시간선택제 공무원 채용 공고, 사이버국가고시센터 11개 부처에서 34명 선발, 5월...","사이버 국가고시센터에서는 시간선택제 공무원을 선발 하는데, 시간선택제 공무원은 통상..."
...,...,...
8556,▲ 면천면 삼웅2리 유재면 전 노인회장의 송덕비 건립기념식이 지난 7일 진행됐다. ...,"면천면 삼웅2리 노인회 회원들이 12년동안 노인회를 이끌어오면서, 노인복지 향상을 ..."
8557,올해 5월 폴란드에서 열리는 국제축구연맹(FIFA) 20세 이하(U-20) 월드컵 ...,FIFA는 오는 25일 올해 5월 폴란드에서 열리는 20세 이하(U-20) 월드컵 ...
8558,[음성]음성군은 지난 19일 군청 상황실에서 유니세프 아동친화도시 인증에 따른 사업...,음성군은 유니세프 아동친화도시 인증에 따른 사업의 안정적인 정착과 내실 있는 추진을...
8559,대구지방식품의약품안전청장 직무대리 가정에서도 HACCP 실천해요 요즘 마트나 슈퍼 ...,요즘 마트나 슈퍼 등에서 위생관리를 더욱 철저히 해 HACCP표시가 된 안전식품들...


#### * dev set도 위와 동일하게 경로명만 변경하여 진행

# 4. 데이터 크기 확인
### train set 크기 (test set 크기)

1. AIhub  

    `신문기사` : 240968 (30121)  
    `사설잡지` : 56055 (7007)  
    `법률문서` : 24029 (3004)  
    `한국어대화요약` : 280000 (35000)  
        - 개인및관계 : 71408 (8926)
        - 미용과건강 : 16520 (2065)
        - 상거래(쇼핑) : 25480 (3185)
        - 시사교육 : 14840 (1855)
        - 식음료 : 29400 (3675)
        - 여가생활 : 35840 (4480)
        - 일과직업 : 20152 (2519)
        - 주거와생활 : 45640 (5705)
        - 행사 : 20720 (2590)
    
      
     
2. Dacon  

    `신문기사` : 34242 (8561)